# Cox PH analysis

### Environment setup

In [ ]:
pip install lifelines

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import math
import scipy.stats as sp
from sklearn.preprocessing import MinMaxScaler
import statsmodels as sm
from ukb_cox_proportional_hazards_utils import compute_is_cancer_at_recruitment, compute_survival_time_with_age_for_label, alcohol_consumption_categorize, smoking_categorize, convert_pvalue_to_asterisks
from outlier_methods import detect_outliers, plot_distribution
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.plotting import plot_lifetimes
from lifelines.calibration import survival_probability_calibration

In [ ]:
s3_path = 's3://ukb-colorectal-cancer/analysis/'

In [ ]:
# Cleaner variable names

rename_mapping={"age": "Age", "sex": "Sex", "ethnicity": "Ethnicity", "townsend": "Townsend", "alcohol": "Alcohol intake", "smoke": "Smoking status", 
"fasted": "Fasting status", "red_meat": "Processed / red meat intake", "oily_fish": "Oily fish intake", "edu_level": "University education", 
"aspirin": "Regular aspirin use", "health_rating": "Self-reported health rating", "family_cancer": "Family history of cancer", "hist_ibd": "IBD history", 
"hist_cvd": "CVD history", "hist_liverbil": "Liver/biliary disease history", "hist_dm": "Diabetes history","met_mins": "Physical activity (MET)", "grip": "Hand grip", "met_rate" : "Metabolic rate",
"trunk_to_leg": "Trunk-to-leg ratio", "waist_to_hip": "Waist-to-hip ratio", "bmi": "BMI", "height": "Height",   "impedance": "Impedance", "sleep_dur": "Duration of sleep", 
"sbp": "Systolic BP", "dbp": "Diastolic BP", "pulse": "Pulse", "hgb": "Haemoglobin", "hct": "Haematocrit %", "wbc": "White blood cell", "rbc": "Red blood cell", 
"plt": "Platelet", "lym": "Lymphocyte %", "mcv": "Mean corpuscular volume", "n_rbc": "Nucleated red blood cell %", "reti": "Reticulocyte %",
"mono": "Monocyte %", "neut": "Neutrophill %", "eos": "Eosinophill %", "baso": "Basophill %", "u_sodium": "Sodium (urine)", 'u_potas': "Potassium (urine)", "u_cr": "Creatinine (urine)",
'apoa':"Apolipoprotein A", 'apob':"Apolipoprotein B",  'chol': "Cholesterol", 'hdl': "HDL cholesterol", 'ldl': "LDL direct", 'tgly': "Triglycerides", 
'urea': "Urea", 'crp': "C-reactive protein",'tprotein': "Total protein",'glu':"Glucose", 'phos': "Phosphate", 'alb':"Albumin",
'alp': "Alkaline phosphatase", 'alt': "Alanine aminotransaminase", 'ast':"Aspartate aminotransferase",'ggt': "Gamma glutamyltransferase", 'urate': "Urate", 
'dbi': "Direct bilirubin", 'tbil': "Total bilirubin",'shbg': "SHBG", 'igf1': "IGF-1", 'vitd': "Vitamin D", 'cysc': "Cystatin C", 'calc':"Calcium", 'hba1c': "HbA1c", 'tst': "Testosterone"}

### Prepare data

In [ ]:
# Get cancer info

df_merged = pd.read_csv("s3://ukb-colorectal-cancer/ukb_crc_longitudinal_v4.csv", low_memory=False)
df_merged["is_cancer-0"] = df_merged.apply(compute_is_cancer_at_recruitment, axis=1)
cancer_prevalent = df_merged[(df_merged["is_cancer-0"]==True)].eid
case_prevalent = df_merged[(df_merged["is_label-0"]==True)].eid
case_incident_0_1 = df_merged[(df_merged["is_label-0"]==False) & (df_merged["is_label-1"]==True)].eid
case_incident_1_2 = df_merged[(df_merged["is_label-0"]==False) & (df_merged["is_label-1"]==False) & (df_merged["is_label-2"]==True)].eid
case_incident_2_3 = df_merged[(df_merged["is_label-0"]==False) & (df_merged["is_label-1"]==False) & (df_merged["is_label-2"]==False) & (df_merged["is_label-3"]==True)].eid
case_incident_3_end = df_merged[(df_merged["label_first_occurred_date"].notna()) & (df_merged.filter(like="is_label").sum(axis=1)==0)].eid
print(f"Number of any cancer occurred before recruitment: {len(cancer_prevalent)}")
print(f"Number of label occurred before recruitment (visit 0): {len(case_prevalent)}")
print(f"Number of incidents occurred after recruitment (visit 0) before visit 1: {len(case_incident_0_1)}")
print(f"Number of incidents occurred after visit 1 before visit 2: {len(case_incident_1_2)}")
print(f"Number of incidents occurred after visit 2 before visit 3: {len(case_incident_2_3)}")
print(f"Number of incidents occurred after visit 3: {len(case_incident_3_end)}")
print(f"Number of participants: {len(df_merged.eid)}")
print(f"Number of any cancer occurred before recruitment: {len(cancer_prevalent)}")
survival_df = df_merged.loc[~df_merged.eid.isin(cancer_prevalent), :]
other_cancer_primary = survival_df[(survival_df["label_first_occurred_date"].isna()) & (survival_df["n_cancer_occurred"]>0)].eid
print(f"Number of participants who developed cancer other than CRC: {len(other_cancer_primary)}")
survival_df = survival_df.loc[~survival_df.eid.isin(other_cancer_primary), :]
other_cancer_concurrent = survival_df[(survival_df["label_first_occurred_date"].notna()) & (survival_df['cancer_first_occurred_date'].notna()) & (survival_df["label_first_occurred_date"] != survival_df["cancer_first_occurred_date"])].eid
print(f"Number of participants who had other concurrent cancers in addition to CRC: {len(other_cancer_concurrent)}")
survival_df = survival_df.loc[~survival_df.eid.isin(other_cancer_concurrent), :]
print(f"Number of participants left: {len(survival_df.eid)}")

In [ ]:
survival_df.label_class.value_counts()

In [ ]:
# Calculate survival as a factor of age of diagnosis

survival_df[["date_lfu", "date_death", "label_first_occurred_date", "visit_date-0"]] = survival_df[["date_lfu", "date_death", "label_first_occurred_date", "visit_date-0"]].apply(pd.to_datetime, errors='coerce') 
censoring_date = pd.to_datetime("29-02-2020", format='%d-%m-%Y')
survival_df[["event_", "age_", "obs_end_date"]] = survival_df.apply(compute_survival_time_with_age_for_label, censoring_date=censoring_date, result_type="expand", axis=1)
print(survival_df.shape)
nonbaseline_cols = [col for col in survival_df.columns if col.endswith(("-1", "-2", "-3"))]
survival_df.drop(nonbaseline_cols, axis="columns", inplace=True)
survival_df.rename(columns=lambda x: x.split("-")[0], inplace=True)
print(survival_df.shape)

In [ ]:
# Recode categorical variables

survival_df["fasted"] = survival_df["fasted"].astype(float)
survival_df["ethnicity"] = survival_df["ethnicity"].apply(lambda x: 'unk' if pd.isnull(x)==True else ('white' if x==1 else 'nonwhite'))
survival_df["met_mins"] = pd.qcut(survival_df.loc[:,"met_mins"], q=5, labels=range(1,6)).values.add_categories('unk').fillna("unk")

survival_df.replace({
    "redmeat_intake": {np.nan: "unk"}, "oily_fish_intake": {np.nan: "unk"}, "famhist_cancer": {np.nan: "unk"}, 
    "edu_university": {np.nan: "unk"}, "regular_aspirin": {np.nan: "unk"}, "crc_screening": {np.nan: "unk"}, 
    "health_rating": {np.nan:"unk"}, "alcohol": {np.nan:"unk"}, "smoke": {np.nan:"unk"}, 
    "diseasehist_ibd": {np.nan:"unk"}, "diseasehist_diabetes": {np.nan:"unk"},
    "diseasehist_cardiovascular": {np.nan:"unk"}, "diseasehist_anyliverbiliary": {np.nan:"unk"}}, inplace=True)

survival_df['ethnicity'] = pd.Categorical(survival_df['ethnicity'], categories = ["white", "nonwhite", "unk"])
survival_df["redmeat_intake"] = pd.Categorical(survival_df["redmeat_intake"], categories = [0,1,2,3,4,5,"unk"]) # could put group 3 as the reference?
survival_df["oily_fish_intake"] = pd.Categorical(survival_df["oily_fish_intake"], categories = [0,1,2,3,4,5,"unk"]) # could put group 2 as the reference?
survival_df["famhist_cancer"] = pd.Categorical(survival_df["famhist_cancer"], categories = [False, True,"unk"])
survival_df["diseasehist_ibd"] = pd.Categorical(survival_df["diseasehist_ibd"], categories = [False, True,"unk"])
survival_df["diseasehist_cardiovascular"] = pd.Categorical(survival_df["diseasehist_cardiovascular"], categories = [False, True,"unk"])
survival_df["diseasehist_diabetes"] = pd.Categorical(survival_df["diseasehist_diabetes"], categories = [False, True,"unk"])
survival_df["diseasehist_anyliverbiliary"] = pd.Categorical(survival_df["diseasehist_anyliverbiliary"], categories = [False, True,"unk"])
survival_df["edu_university"] = pd.Categorical(survival_df["edu_university"], categories = [False, True,"unk"])
survival_df["regular_aspirin"] = pd.Categorical(survival_df["regular_aspirin"], categories = [False, True,"unk"])
survival_df["crc_screening"] = pd.Categorical(survival_df["crc_screening"], categories = [False, True,"unk"])
survival_df["health_rating"] = pd.Categorical(survival_df["health_rating"], categories = [4,3,2,1,"unk"]) # 1-excellent, 2-good, 3-fair, 4-poor # could put group 2 as the reference?
survival_df["alcohol"] = pd.Categorical(survival_df["alcohol"], categories = [0,1,2,3,4,5,6,"unk"])
survival_df['smoke'].replace(4,'unk', inplace=True)
survival_df["smoke"] = pd.Categorical(survival_df["smoke"], categories = [0,1,2,3,"unk"])
survival_df["met_mins"] = pd.Categorical(survival_df["met_mins"], categories = [1,2,3,4,5,"unk"])

In [ ]:
# Select columns

selected_cols = ["age", "sex", "ethnicity", "townsend", "alcohol", "smoke", "fasted", 
                 "redmeat_intake", "oily_fish_intake", "famhist_cancer", "edu_university", "regular_aspirin",  "health_rating", 
                 "diseasehist_ibd", 'diseasehist_cardiovascular', 'diseasehist_diabetes','diseasehist_anyliverbiliary', "met_mins","hgrip", "tlr", "whr", "bmi", "height", "met_rate", "impedance", "sleep_dur", 
                 "sbp", "dbp", "pulse", "hgb", "hct", "wbc", "rbc", "plt", "lym", "mcv", "mono", "neut", "eos", "baso", "n_rbc", "reti",
                 "u_sodium", 'u_potas', "u_cr",'apoa', 'apob',  'chol', 'hdl', 'ldl', 'tgly', 'urea', 'crp','tprotein',
                 'glu', 'phos', 'alb', 'alp', 'alt', 'ast', 'ggt', 'urate', 'd_bil', 't_bil','shbg', 'igf1', 'vitd', 'cysc', 'calc',  'hba1c', 'tst'] 
# Removed "crc_screening",

full_df = survival_df.loc[:, ["event_", "age_"] + [col for col in survival_df.columns if col.split("-")[0] in selected_cols]]

In [ ]:
# Remove NaNs

print(full_df.shape)
print(f"Number of rows with missing values: {full_df.isna().any(axis=1).sum()}")
full_df.isna().sum(axis=0).sort_values(ascending=False).head(20)
full_df.dropna(inplace=True)
print(full_df.shape)
print(f"Number of rows with missing values: {full_df.isna().any(axis=1).sum()}")

full_df.event_.value_counts().to_dict()

In [ ]:
# Remove outliers

continuous_vars = ["hgrip", "tlr", "whr",  "height", "met_rate", "impedance", "sleep_dur", 
                 "sbp", "dbp", "pulse", "bmi", "hgb", "hct", "wbc", "rbc", "plt", "lym", "mcv", "mono", "neut", "eos", "baso", "n_rbc", "reti",
                 "u_sodium", 'u_potas', "u_cr",'apoa', 'apob',  'chol', 'hdl', 'ldl', 'tgly', 'urea', 'crp','tprotein',
                 'glu', 'phos', 'alb', 'alp', 'alt', 'ast', 'ggt', 'urate', 'd_bil', 't_bil','shbg', 'igf1', 'vitd', 'cysc', 'calc',  'hba1c', 'tst']

outliers = []
for i, col in enumerate(continuous_vars):
    outliers_ = detect_outliers(full_df, col, method="percentile", percentile_threshold=0.001) # 0.005
    outliers += list(outliers_)
    
outliers = np.unique(outliers)
print(f"Number of outliers: {len(outliers)}")

full_df.drop(outliers, axis='index', inplace=True)
print(full_df.shape)

In [ ]:
full_df.event_.value_counts().to_dict()

In [ ]:
full_df.columns = ['event_', 'age_', 'sex', 'townsend', 'ethnicity', 'bmi', 'wbc', 'rbc',
       'hgb', 'hct', 'plt', 'lym', 'u_cr', 'u_potas', 'u_sodium', 'apoa',
       'apob', 'urea', 'chol', 'crp', 'cysc', 'hdl', 'igf1', 'ldl', 'shbg',
       'tst', 'tprotein', 'tgly', 'vitd', 'pulse', 'dbp', 'sbp', 'age',
       'oily_fish', 'health_rating', 'height',
       'family_cancer', 'hist_cvd', 'hist_dm', 'hist_ibd', 'hist_liverbil', 'edu_level',
       'aspirin', 'red_meat', 'fasted', 'alcohol', 'smoke',
       'sleep_dur', 'met_mins', 'met_rate', 'impedance', 'mcv', 'mono', 'neut',
       'eos', 'baso', 'n_rbc', 'reti', 'alb', 'alp', 'alt', 'ast', 'dbi',
       'calc', 'ggt', 'glu', 'hba1c', 'phos', 'tbil', 'urate', 'grip', 'waist_to_hip',
       'trunk_to_leg']

In [ ]:
full_df.to_csv(s3_path+'crc_dataset_cox.csv', index=False)

### Forward feature selection

##### Fitting each variable separately on the training dataset, and selecting variables that have a p-value<0.10 (more liberal threshold)

In [ ]:
df = pd.read_csv(s3_path+'crc_dataset_cox.csv', dtype={'sex':'category','fasted':'category','ethnicity':'category', 'oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_cvd':'category','hist_dm':'category', 'hist_ibd':'category','hist_liverbil':'category',
                                                      'edu_level':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'},low_memory=False)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.2, random_state=1, stratify=df["event_"])

X_train.to_csv(s3_path+'crc_dataset_cox_train.csv', index=False)
X_test.to_csv(s3_path+'crc_dataset_cox_test.csv', index=False)

In [ ]:
cph = CoxPHFitter()
cols = df.drop(['event_', 'age_'], axis=1).columns.to_list()

mdl_name = []
var_keep = []
c_idx = []
aic = []
p_val = []
var_hr = []
var_se = []
var_pval = []

In [ ]:
for c in cols:
    cph.fit(X_train, duration_col='age_', event_col='event_', formula=c, show_progress=False) 
    mdl_name.append(c)
    c_idx.append(round(cph.concordance_index_,4))
    aic.append(round(cph.AIC_partial_,2))
    summary = cph.summary["p"].to_dict()
    p_val.append(round(min(list(summary.values())),3))
    print('Model:', c, 'C-index:', c_idx[-1],'AIC:', aic[-1], 'p:', p_val[-1])
    if p_val[-1] < 0.1:
        var_keep.append(c)
        var_hr.append(cph.summary["coef"][0])
        var_se.append(1.96*(cph.summary["se(coef)"][0]))
        var_pval.append(p_val[-1])

In [ ]:
# Get univariate parameters for the plots

var_keep = [v for _, v in sorted(zip(var_hr, var_keep))] # put in descending order
var_se = [v for _, v in sorted(zip(var_hr, var_se))]
var_pval = [v for _, v in sorted(zip(var_hr, var_pval))]
var_hr.sort()

a = []
for v in var_keep:
    if '[' in v:
        v = v[:v.index('[')]
    a.append(v)
var_keep = a
var_names = [rename_mapping[v] for v in var_keep]

sz = []
for p in var_pval:
    if p > 0.05:
        sz.append(5) 
    elif p < 0.001:
        sz.append(60)
    elif p < 0.01:
        sz.append(40)
    elif p < 0.05:
        sz.append(20)     


In [ ]:
# Plot log(HR) of the selected features in univariate model

theme = matplotlib.colors.LinearSegmentedColormap.from_list("", ['blue','gainsboro','red'])
a = [theme(1. * i / len(var_keep)) 
    for i in range(len(var_keep))]
a = [list(i[:3]) for i in a]

fig, ax = plt.subplots(figsize=(3,12))
for i in range(len(var_hr)):
    plt.plot([var_hr[i]-var_se[i], var_hr[i]+var_se[i]],[i, i], color=a[i])
ax.set_yticks(np.arange(len(var_hr)))
ax.set_yticklabels(var_keep)
plt.xlabel('log(HR) 95% CI')
plt.axvline(x = 0, color = 'silver', linestyle='--')
plt.scatter(var_hr, range(len(var_hr)), s=60, c=a) # plt.scatter(hr, range(len(hr)), s=60, c=a)
ax.set_yticklabels(var_names)
plt.savefig('./figures/paper_all_hazard_ratios.jpg', dpi=150) 
plt.show()


In [ ]:
var_keep = var_keep + ['event_', 'age_']
X_train = X_train[var_keep].copy(deep=True)
X_test = X_test[var_keep].copy(deep=True)

X_train.to_csv(s3_path+'crc_dataset_cox_train_fs.csv', index=False)
X_test.to_csv(s3_path+'crc_dataset_cox_test_fs.csv', index=False)

### VIF - Remove correlated features

##### Find and remove variables correlated with each other, to reduce multicollinearity, and clarify the contribution of each predictor to the model based on variance inflation factor >10.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv(s3_path+'crc_dataset_cox_train_fs.csv', dtype={'sex':'category','fasted':'category','oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_dm':'category', 'hist_liverbil':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'}, low_memory=False)

In [ ]:
df.replace('unk',np.NaN, inplace=True)
df.replace(['False', 'True'],[0,1], inplace=True)

for c in df:
    df[c] = df[c].astype(float)
df.dropna(inplace=True)

In [ ]:
# Calculate VIF

cols = ['sex', 'bmi', 'wbc', 'rbc', 'hgb', 'lym', 'u_cr', 'u_potas',
       'u_sodium', 'urea', 'chol', 'crp', 'hdl', 'igf1', 'shbg',
       'tgly', 'vitd', 'pulse', 'dbp', 'age', 'oily_fish',
       'health_rating', 'family_cancer', 'hist_dm', 'hist_liverbil',
       'aspirin', 'red_meat', 'fasted', 'alcohol', 'smoke', 'met_mins',
        'mono', 'baso', 'reti', 'alt',
       'calc', 'ggt', 'phos', 'tbil', 'urate', 'grip', 'waist_to_hip',
       'trunk_to_leg']
var_keep = cols

# (CHOL - ldl) (LYM - neut) (RBC - hct) (TBIL - dbi) (HDL - apoa) (SEX -  tst, impedance, met_rate,height)

vif_df = pd.DataFrame()
vif_df["variable"] = df[cols].columns

X = df[cols].copy(deep=True)
X['intercept'] = 1
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_df['vif'] = vif[:-1]

In [ ]:
vif_df.sort_values(by='vif',ascending=False)[:15]

In [ ]:
# Plot dendrogram

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, set_link_color_palette
from scipy.spatial.distance import squareform

cols = df.drop(['age_','event_'],axis=1).columns
col_names = [rename_mapping[c] for c in cols]

set_link_color_palette(['darkmagenta','navy','royalblue','lightseagreen','limegreen','gold','darkorange','orangered']) # 9
corrs = df.drop(['age_','event_'], axis=1).corr()
plt.figure(figsize=(2,10))
dissimilarity = 1 - abs(corrs)
Z = linkage(squareform(dissimilarity), 'ward')

R1 = dendrogram(Z, labels=cols, orientation='left', color_threshold=1.12, leaf_font_size=10, count_sort='descending', above_threshold_color='silver'); # leaf_rotation=90, distance_sort='descending',);
R = dendrogram(Z, labels=col_names, orientation='left', color_threshold=1.12, leaf_font_size=10, count_sort='descending', above_threshold_color='silver') #distance_sort='descending', 
plt.show()

In [ ]:
# Plot r-map in the leaf order of the dendrogram

new_order = R1['ivl']
new_order.reverse()
col_names = [rename_mapping[c] for c in new_order]

corrs = df[new_order].corr()
pval = df[new_order].corr(method=lambda x, y: sp.pearsonr(x, y)[1]) - np.eye(*corrs.shape)

mask = np.triu(np.ones_like(corrs, dtype=bool))
plt.figure(figsize=(14,14))
sns.heatmap(corrs, annot=False, annot_kws = {'size':9},fmt='.2f', mask=mask, xticklabels=col_names, yticklabels=col_names, \
            square=True, cbar_kws={"shrink": 0.5}, cmap='bwr', vmin=-0.6, vmax=0.6).set(title='Intercorrelations - rmap')

#plt.savefig('./figures/biomarker_rmap_everyone.jpg', dpi=150) 
plt.show()

In [ ]:
df = pd.read_csv(s3_path+'crc_dataset_cox_train_fs.csv', dtype={'sex':'category','fasted':'category','oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_dm':'category', 'hist_liverbil':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'},low_memory=False)
var_keep = var_keep + ['event_', 'age_']
df = df[var_keep].copy(deep=True)
df.to_csv(s3_path+'crc_dataset_cox_train_fs_vif.csv', index=False)

df = pd.read_csv(s3_path+'crc_dataset_cox_test_fs.csv', dtype={'sex':'category','fasted':'category','oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_dm':'category', 'hist_liverbil':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'},low_memory=False)
df = df[var_keep].copy(deep=True)
df.to_csv(s3_path+'crc_dataset_cox_test_fs_vif.csv', index=False)

### Backward elimination

##### Start by fitting the all of the selected features, and remove features that don't significantly contribute to the model using p > 0.05, starting with the lowest significance in an ascending order.

In [ ]:
df = pd.read_csv(s3_path+'crc_dataset_cox_train_fs_vif.csv', dtype={'sex':'category','fasted':'category','oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_dm':'category', 'hist_liverbil':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'}, low_memory=False)
df["health_rating"] = pd.Categorical(df["health_rating"], categories = ['4','3','2','1',"unk"])

In [ ]:
# Initiate

cols = df.drop(['event_','age_'],axis=1).columns # all features from the forward feature selection step
mdl_cols = cols.to_list()
mdl_formula = " + ".join(mdl_cols)

cph = CoxPHFitter()
cph.fit(df, duration_col='age_', event_col='event_', formula=mdl_formula, show_progress=False) 
print('Model:', mdl_formula)
print('-- C-index:', round(cph.concordance_index_,6),'AIC:', round(cph.AIC_partial_,2))

summary = cph.summary["p"].to_dict()
mdl_vars = list(summary.keys())
mdl_pvals = np.array(list(summary.values()))

vnames = []
min_p = []
for v in mdl_vars: # get the minimum p-value of each variable
    if '[' in v:
        i = v.index('[')
        vname = v[:i]
    else:
        vname = v
    vnames.append(vname)   
    idx = [mdl_vars.index(i) for i in mdl_vars if i.startswith(vname)]
    min_p.append(np.min(mdl_pvals[idx]))

while np.max(min_p) > 0.0499: # whilst there are non-significant variables not accepting marginal effects
    
    idx = np.argmax(min_p)
    mdl_cols.remove(vnames[idx]) # remove that variable from the list and rerun the model
    print('Removing', vnames[idx])
    
    mdl_formula = " + ".join(mdl_cols)

    cph.fit(df, duration_col='age_', event_col='event_', formula=mdl_formula, show_progress=False) 
    print('Model:', mdl_formula)
    print('-- C-index:', round(cph.concordance_index_,6),'AIC:', round(cph.AIC_partial_,2))
    summary = cph.summary["p"].to_dict()
    mdl_vars = list(summary.keys())
    mdl_pvals = np.array(list(summary.values()))

    vnames = []
    min_p = []
    for v in mdl_vars:
        if '[' in v:
            i = v.index('[')
            vname = v[:i]
        else:
            vname = v
        vnames.append(vname)   
        idx = [mdl_vars.index(i) for i in mdl_vars if i.startswith(vname)]
        min_p.append(np.min(mdl_pvals[idx]))
    

In [ ]:
cph.print_summary(decimals=4)

In [ ]:
# Test performance on the test set

df_test = pd.read_csv(s3_path+'crc_dataset_cox_test_fs_vif.csv', dtype={'sex':'category','fasted':'category','oily_fish':'category', 'health_rating':'category', 'family_cancer':'category',
                                                      'hist_dm':'category', 'hist_liverbil':'category','aspirin':'category','red_meat':'category','alcohol':'category',
                                                      'smoke':'category', 'met_mins':'category'}, low_memory=False)
cph = CoxPHFitter()
cph.fit(df_test, duration_col='age_', event_col='event_', formula=" + ".join(mdl_cols), show_progress=False) 
print('-- C-index:', round(cph.concordance_index_,6),'AIC:', round(cph.AIC_partial_,2))

In [ ]:
mdl_cols = mdl_cols + ['event_', 'age_']
df = df[mdl_cols].copy(deep=True)
df.to_csv(s3_path+'crc_dataset_cox_train_fs_vif_bs.csv', index=False)

### Plot results

In [ ]:
df = pd.read_csv(s3_path+'crc_dataset_cox_train_fs_vif_bs.csv', dtype={'sex':'category','health_rating':'category', 'family_cancer':'category',
                                                      'aspirin':'category','alcohol':'category','smoke':'category'}, low_memory=False)

In [ ]:
cph = CoxPHFitter()
cols = df.drop(['event_','age_'],axis=1).columns 
mdl_cols = cols.to_list()

cph.fit(df, duration_col='age_', event_col='event_', formula=" + ".join(mdl_cols), show_progress=False)
summary = cph.summary["p"].to_dict()
mdl_vars = list(summary.keys())
mdl_pvals = np.array(list(summary.values()))

In [ ]:
# Default plot from lifelines log(HR) plot

plt.figure(figsize=(5,5))
ax = cph.plot()# hazard_ratios=True
plt.show()

In [ ]:
# HR plot

plt.figure(figsize=(5,5))
ax = cph.plot(hazard_ratios=True)
plt.show()

In [ ]:
# Get stats for the plots

hr = cph.summary["coef"].to_dict() # get values to plot
mdl_vars = list(hr.keys())
hr = cph.summary["coef"]
se = cph.summary["se(coef)"]
pval = cph.summary["p"]

idx = [i for i in range(len(pval)) if pval[i] < 0.05  and 'T.unk' not in mdl_vars[i]] # threshold by p
pval = [pval[i] for i in idx]
hr = [hr[i] for i in idx]
se = np.array([1.96*se[i] for i in idx])
mdl_vars = [mdl_vars[i] for i in idx]

mdl_vars = [v for _, v in sorted(zip(hr, mdl_vars))] # put in descending order
se = [v for _, v in sorted(zip(hr, se))]
pval = [v for _, v in sorted(zip(hr, pval))]
hr.sort()

a = []
for v in mdl_vars:
    if '[' in v:
        v = v[:v.index('[')]
    a.append(v)
mdl_vars = a
var_names = [rename_mapping[v] for v in mdl_vars]

sz = []
for p in pval:
    if p > 0.05:
        sz.append(5) 
    elif p < 0.001:
        sz.append(60)
    elif p < 0.01:
        sz.append(40)
    elif p < 0.05:
        sz.append(20)     

In [ ]:
# Plot using a risk gradient

theme = matplotlib.colors.LinearSegmentedColormap.from_list("", ['blue','gainsboro','red'])
a = [theme(1. * i / len(mdl_vars)) 
    for i in range(len(mdl_vars))]
a = [list(i[:3]) for i in a]

fig, ax = plt.subplots(figsize=(5,5))
for i in range(len(hr)):
    plt.plot([hr[i]-se[i], hr[i]+se[i]],[i, i], color=a[i])
ax.set_yticks(np.arange(len(hr)))
ax.set_yticklabels(mdl_vars)
plt.xlabel('log(HR) 95% CI')
plt.axvline(x = 0, color = 'silver', linestyle='--')
plt.scatter(hr, range(len(hr)), s=60
            , c=a) # plt.scatter(hr, range(len(hr)), s=60, c=a)
ax.set_yticklabels(var_names)
plt.show()

In [ ]:
df = pd.read_csv(s3_path+'crc_dataset_cox_train_fs_vif_bs.csv', dtype={'sex':'category','health_rating':'category', 'family_cancer':'category',
                                                      'aspirin':'category','alcohol':'category','smoke':'category'}, low_memory=False)

In [ ]:
# Changing categories in string format to integers (e.g. '2' to 2)

df.replace(['False', 'True','0','1','2','3','4','5','6','unk'],[0,1,0,1,2,3,4,5,6,9], inplace=True)
df['health_rating'] = pd.Categorical(df['health_rating'], categories = [1,2,3,4,9])
df['family_cancer'] = pd.Categorical(df['family_cancer'], categories = [0,1,9])
df['aspirin'] = pd.Categorical(df['aspirin'], categories = [0,1,9])
df['alcohol'] = pd.Categorical(df['alcohol'], categories = [0,1,2,3,4,5,6,9])
df['smoke'] = pd.Categorical(df['smoke'], categories = [0,1,2,3,4,9])

In [ ]:
# Kaplan Meier survival plots

cph = CoxPHFitter()
cph.fit(df, duration_col='age_', event_col='event_', show_progress=False)

cph.plot_partial_effects_on_outcome(covariates=['waist_to_hip'], values=[0.7,0.8,0.9,1,1.1], cmap='viridis', plot_baseline=False) 
plt.xlim([60,85])
plt.show()


In [ ]:
plt.figure(figsize=(5,5))
cph.plot_partial_effects_on_outcome(covariates=['alcohol'], values=[0,1,2,3,4,5,6], cmap='viridis', plot_baseline=False,figsize=(5,5))
plt.xlim([60,85])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
cph.plot_partial_effects_on_outcome(covariates=['sex'], values=[0,1], cmap='viridis', plot_baseline=False,figsize=(5,5)) 
plt.xlim([60,85])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
cph.plot_partial_effects_on_outcome(covariates=['family_cancer'], values=[0,1], cmap='coolwarm', plot_baseline=False,figsize=(5,5)) 
plt.xlim([60,85])
plt.show()